In [1]:
import numpy as np
import cv2

In [2]:
def cosine_similarity(x, y):
    numerator = np.dot(x,y)
    denominator = np.linalg.norm(x)*np.linalg.norm(y)

    return numerator/denominator

In [3]:
def window_based_matching(left_img, right_img, disparity_range, kernel_size=5, save_result=True):
    # Đọc và chuyển đổi ảnh trái, phải sang dạng grayscale
    left = cv2.imread(left_img, 0).astype(np.float32)
    right = cv2.imread(right_img, 0).astype(np.float32)
    
    height, width = left.shape[:2]
    
    # Tạo bản đồ disparity rỗng
    depth = np.zeros((height, width), np.uint8)
    kernel_half = kernel_size // 2
    scale = 3
    
    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            # Tìm j với chi phí có giá trị tối ưu nhất
            disparity = 0
            cost_optimal = -1

            for j in range(disparity_range):
                d = x - j
                cost = -1
                if (d - kernel_half) > 0:
                    wp = left[(y - kernel_half):(y + kernel_half) + 1, (x - kernel_half):(x + kernel_half) + 1]
                    wqd = right[(y - kernel_half):(y + kernel_half) + 1, (d - kernel_half):(d + kernel_half) + 1]

                    wp_flattened = wp.flatten()
                    wqd_flattened = wqd.flatten()

                    cost = cosine_similarity(wp_flattened, wqd_flattened)

                    if cost > cost_optimal:
                        cost_optimal = cost
                        disparity = j

            # Gán giá trị độ sâu tại (y, x) bằng j (disparity)
            # Nhân với hệ số tỉ lệ để hiển thị
            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        # Lưu kết quả
        cv2.imwrite('result/window_based_cosine_similarity.png', depth)
        cv2.imwrite('result/window_based_cosine_similarity_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print('Done.')
    return depth

In [4]:
window_based_matching("Aloe/Aloe_left_1.png","Aloe/Aloe_right_3.png",disparity_range = 64, kernel_size=5)

Saving result...
Done.


array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,  48,   0,   0],
       ...,
       [  0,   0,   0, ..., 126,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)